# Simulated Annealing for Mars Lander Episode-1 

### Import modules and Utilities

In [1]:
from typing import List, Optional, Callable, Tuple, Dict, Union  # OneNote2021 #_python_ #_typing_
GENE = List[int]  # (rotate (-15 ~ +15),power (-1, 0, +1)) 避免無用的值域，下給 Web site 的 command 要轉換成 (rotate(-90 ~ +90), power(0,1,2,3,4))
GENOME = List[GENE]  # Genome 是一個 list 裡面都是 int
STATE = Dict[str,Union[int,float]]  # x,y,hspeed,power,.. etc 

import numpy as np
import re, math, random, pdb, json, copy
import peforth, IPython
peforth.bps = [i for i in range(100)]  # 預設有這麼多 Breakpoint ID
def bp(id=None,locals=None):
    # Breakpoint ID 不能超過 peforth.bps 保留，超過的無效。
    if id==None: 
        id = 0
        prompt='bp> '
    else:
        prompt="{}>".format(id)
    if id in peforth.bps: peforth.push(locals).ok(prompt, cmd="to _locals_")
peforth.bp = bp
peforth.push(bp).dictate("py: vm.bp=pop()")  # [ ] 忘了 vm.bp 有何用途？
peforth.dictate('''
    import IPython
    : paste py> IPython.lib.clipboard.win32_clipboard_get() tib.insert ;
        // ( ... -- ... ) 執行 clipboard 裡的內容，jupyternotebook 進了 peforth prompt 特別需要此功能。
    \ ------ breakpoint ------------------------------------------------------------
    : bp ." Usage: peforth.bp(11,locals())  # drop a breakpoint with ID=11" cr ;
    : bl // ( -- ) List all breakpoints
        __main__ :> peforth.bps 
        <py>
        bps = pop()
        print('Disabled breakpoints:')
        for i in range(len(bps)):
            if not bps[i]: 
                print(i, end=' ')
        print(); print('Enabled breakpoints:')
        count = 0
        for i in range(len(bps)):
            if bps[i]: 
                print(i, end=' ')
                count += 1
        print(); print('Enabled breakpoints count: {}/{}'.format(count,len(bps)))
        </py> cr ;
        /// Breakpoint commands:
        ///	  bl  - list all breakpoints (capital BL is white space) 
        ///	  be  - enable breakpoints, e.g. be 1 2 3 
        ///	  bd  - disable breakpoints, e.g. bd 1 2 3 
        ///	  be* - enable all breakpoints
        ///	  bd* - disable all breakpoints 
        /// Also: 
        ///   for i in [11,22,33]: peforth.bps[i]=0	 # disable breakpoints 11,22,33 
        ///   for i in [11,22,33]: peforth.bps[i]=i	 # enable  breakpoints 11,22,33 
    
    : bd // ( <1 2 3 4...> -- ) Disable listed breakpoints 
        CR word ( line ) __main__ :> peforth.bps ( line bps )
        <py>
        line, bps = pop(1), pop(0)
        points = map(int, line.split(' '))
        for i in points: bps[i] = 0
        </py> ; 
        ' bl :> comment last :: comment=pop(1)
    
    : be // ( <1 2 3 4...> -- ) Enable listed breakpoints 
        CR word ( line ) __main__ :> peforth.bps ( line bps ) 
        <py>
        line, bps = pop(1), pop(0)
        points = map(int, line.split(' '))
        for i in points: bps[i] = i
        </py> ; 
        ' bl :> comment last :: comment=pop(1)
    
    : bd* // ( -- ) Disable all breakpoints 
        __main__ :> peforth.bps	 ( bps ) 
        <py>
        bps = pop()
        for i in range(len(bps)): bps[i] = 0
        </py> ;
        ' bl :> comment last :: comment=pop(1)
    
    : be* // ( -- ) Enable all breakpoints 
        __main__ :> peforth.bps	 ( bps ) 
        <py>
        bps = pop()
        for i in range(len(bps)): bps[i] = i
        </py> ;
        ' bl :> comment last :: comment=pop(1)
    \ ------ xtack ------------------------------------------------------------
    [] value xstack xstack py: vm.xstack=pop() // ( -- array ) The xstack. 掛進 vm 就可以直接以 py> xstack 取用。
    : x@ xstack :> [-1] ; // ( -- n ) Get TOS of the xstack
    : x> xstack :> pop() ; // ( -- n ) Pop the xstack
    : >x xstack :: append(pop()) ; // ( n -- ) Push n into the xstack
    : .sx xstack . ; // ( -- ) List xstack 
    : xdrop x> drop ; // ( X: ... a -- X: ... ) drop xstack 
    : xdropall [] to xstack ; // ( X: ... -- X: empty ) clear xstack 
    ''')
%f version ==>


reDef unknown
reDef -->
p e f o r t h    v1.26
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.

version ==>
 1.26 (<class 'str'>)


In [2]:
# 把 genome 轉成 Web 接受的 command list 
def genome_to_command_list(genome:GENOME) -> List[List[int]] :
    p = 0  # initial power is always 0 
    a = [] # result command array of [rotate, thrust] paris 
    for i in genome :
        if i[1] == -1 :
            p = max(p-1, 0) 
            a.append([0, p])
            pass
        elif i[1] == 0 :
            a.append([0, p])
            pass
        elif i [1] == 1 :
            p = min(p+1, 4) 
            a.append([0, p])
            pass
        else:
            pass
    return a

%f genome_to_command_list :> ([[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,-1],[0,-1],[0,-1],[0,-1],[0,-1]]) -->

genome_to_command_list :> ([[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,-1],[0,-1],[0,-1],[0,-1],[0,-1]]) --> [[0, 1], [0, 2], [0, 3], [0, 4], [0, 4], [0, 4], [0, 3], [0, 2], [0, 1], [0, 0], [0, 0]] (<class 'list'>)


In [3]:
# Get distance of two genomes where genomes are vectors  
def distance(g1:GENOME, g2:GENOME):
    def vector(genome): 
        a = [i[1] for i in genome]
        assert len(a)<=200, 'Input genome length too long {}'.format(len(a))
        for i in range(200-len(a)):
            a.append(0)
        return np.array(a)
    return np.linalg.norm(vector(g1)-vector(g2))

%f distance :> ([[0,0],[0,0]],[[0,1],[0,1]]) -->

distance :> ([[0,0],[0,0]],[[0,1],[0,1]]) --> 1.4142135623730951 (<class 'numpy.float64'>)


# Episode-1 

In [4]:
# 最基本的定義 

gravity = -3.711        # constant on Mars
g = gravity             # 到了二維以上，重力需轉成向量

class State(STATE):

    def __init__(self,step=0,x=0,y=0,hspeed=0,
            vspeed=0,fuel=0,angle=0,power=0,force=0):
        self.step   = step
        self.x      = x
        self.y      = y
        self.hspeed = hspeed
        self.vspeed = vspeed
        self.fuel   = fuel
        self.angle  = angle
        self.power  = power
        self.force  = force

    def next(self, angle:int, power:int):  
        # input angle,power 都是 offset 分別屬 -15~15 以及 -1,0,+1 
        assert power in [-1, 0, 1], 'expected input power is an offset [-1, 0, +1], given {}'.format(power)
        assert angle in range(-15,16), 'input angle range [-15:16], given {}'.format(angle)
        self.step   += 1
        self.angle  += angle
        self.angle   = -90 if self.angle < -90 else self.angle
        self.angle   =  90 if self.angle > 90 else self.angle
        self.power  += power
        self.power   = 0 if self.power < 0 else self.power
        self.power   = 4 if self.power > 4 else self.power
        self.fuel   -= self.power
        self.x       = self.x       # episode-1 不變
        self.hspeed  = self.hspeed  # episode-1 不變
        self.force   = g + self.power
        self.y       = self.y + self.vspeed + (1/2)*self.force
        self.vspeed += self.force
        return self  # 傳回值只是方便測試，可以這樣寫： %f state :> next(0,3.711) -->

    def __str__(self):  
        return str(self.__dict__)

fuel0 = 500             # episode-1 
state0 = State(0,0,2500,0,0,fuel0,0,0,0)  # episode-1 的起點 state


In [5]:
# 建置 command lists 錄製好幾種狀況方便實驗

sample_genomes = {"自由落體":[],"安全降落":[], "飛向太空":[], "差點成功":[], "更成功":[]}  
state_list = {"自由落體":{},"安全降落":{}, "飛向太空":{}, "差點成功":{}, "更成功":{}}  

# 自由落體
state = State(**state0.__dict__)  # init for ecah genome 
for i in range(36):
    state.next(0,0);
    sample_genomes["自由落體"].append([0,0])
state_list["自由落體"]=state  # state 是獨立的，不需要 deep copy 

# 安全降落
state = State(**state0.__dict__)  # init for ecah genome 
for i in range(9):
    state.next(0,0);
    sample_genomes["安全降落"].append([0,0])
for i in range(9,91):
    state.next(0,1);
    sample_genomes["安全降落"].append([0,1])
state_list["安全降落"]=state

# 飛向太空
state = State(**state0.__dict__)  # init for ecah genome 
for i in range(83):
    state.next(0,1);
    sample_genomes["飛向太空"].append([0,1])
state_list["飛向太空"]=state

# 更成功
state = State(**state0.__dict__)  # init for ecah genome 
for i in range(9):
    state.next(0,0);
    sample_genomes["更成功"].append([0,0])
for i in range(9,81):
    state.next(0,1);
    sample_genomes["更成功"].append([0,1])
for i in range(7):
    state.next(0,-1);
    sample_genomes["更成功"].append([0,-1])
state_list["更成功"]=state

%f sample_genomes :> ["自由落體"] --> \ genome_to_command_list :> (pop()) -->
%f sample_genomes :> ["安全降落"] --> \ genome_to_command_list :> (pop()) -->
%f sample_genomes :> ["飛向太空"] --> \ genome_to_command_list :> (pop()) -->
%f sample_genomes :> ["更成功"]  -->  \ genome_to_command_list :> (pop()) -->

%f state_list :> ["自由落體"] -->  
%f state_list :> ["安全降落"] -->  
%f state_list :> ["飛向太空"] -->  
%f state_list :> ["更成功"]   -->  


sample_genomes :> ["自由落體"] --> [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]] (<class 'list'>)
sample_genomes :> ["安全降落"] --> [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1],

In [6]:
# 改寫 for Mars Lander 乍看以為要頭痛一會兒，其實 Mars Lander 的 genome 很簡單，就是一串 (int,int) 
# pairs 而已。前面基礎討論有提到，以 SA 而言就是 mutate 某一個 gene 也就是 command pair 而已。但是考慮 Mars
# Lander 的限制條件，以 epidode-1 而言，command 的改變要做的決定是 -1 0 +1 其中之一才對，而非 0~4. 

# 長度的問題，
# 太長而無效的 genes 要 cut 掉。
# 長度不夠時直接 copy 最後一個 gene 頂替即可。
# 會發現 genome 太長或不夠的是 simulator 那就是由它處理。

# 1. 避免無效的 neighbour 例如前一 power 是 0 或 4 卻下 -1 或 1 就是無效的，這個 neighbour 就浪費了。 
# 2. 第一個 gene 的 power 不能下 -1 只能下 0 或 +1 , 將來 angle 也有一樣的限制。

def get_random_neighbour(genome: GENOME) -> None :
    neighbour = copy.deepcopy(genome)  # 利用 [:] slice copy 不行！
    commands = genome_to_command_list(neighbour)  # 絕對 command, 用來檢查前一狀態避免產生無效的 neighbour 
    
    while True:  # 試到不一樣了才出來
        idx = random.randint(0,len(genome)-1) # 選 1 個 gene 
        cmd = random.randint(-1, 1)  # -1, 0, +1 當中選 1 個 thrust command (episode-1)
        if idx == 0 and cmd == -1 : continue  # 一開始的 power 就是 0 再下 -1 無效
        if idx >= 1 and cmd == -1 and commands[idx-1][1] == 0: continue  # 前一個 power 就已經是 0 又下 -1 無效
        if idx >= 1 and cmd ==  1 and commands[idx-1][1] == 4: continue  # 前一個 power 就已經是 4 又下 +1 無效
        if neighbour[idx][1] == cmd: continue  # 沒有變化當然無效
        else: 
            neighbour[idx][1] = cmd
            break
            
    return neighbour
%f get_random_neighbour :> ([[0,0],[0,0]]) ( 都不可能出現 -1 ) -->    
%f get_random_neighbour :> ([[0,0],[0,1]]) ( 都不可能出現 -1 ) -->    
%f get_random_neighbour :> ([[0,1],[0,1],[0,1],[0,1],[0,1]]) --> \ gene0 只有 0 或 1 不會有 -1, gene1 視 gene0 而定     
%f get_random_neighbour :> ([[0,1],[0,1],[0,1],[0,1],[0,1]]) --> \ gene0 只有 0 或 1 不會有 -1, gene1 視 gene0 而定     
%f get_random_neighbour :> ([[0,1],[0,1],[0,1],[0,1],[0,1]]) --> \ gene0 只有 0 或 1 不會有 -1, gene1 視 gene0 而定     
%f get_random_neighbour :> ([[0,1],[0,1],[0,1],[0,1],[0,1]]) --> \ gene0 只有 0 或 1 不會有 -1, gene1 視 gene0 而定     
                             

get_random_neighbour :> ([[0,0],[0,0]]) ( 都不可能出現 -1 ) --> [[0, 0], [0, 1]] (<class 'list'>)
get_random_neighbour :> ([[0,0],[0,1]]) ( 都不可能出現 -1 ) --> [[0, 1], [0, 1]] (<class 'list'>)
get_random_neighbour :> ([[0,1],[0,1],[0,1],[0,1],[0,1]]) --> [[0, 1], [0, 0], [0, 1], [0, 1], [0, 1]] (<class 'list'>)
get_random_neighbour :> ([[0,1],[0,1],[0,1],[0,1],[0,1]]) --> [[0, 1], [0, 0], [0, 1], [0, 1], [0, 1]] (<class 'list'>)
get_random_neighbour :> ([[0,1],[0,1],[0,1],[0,1],[0,1]]) --> [[0, 1], [0, 1], [0, 1], [0, -1], [0, 1]] (<class 'list'>)
get_random_neighbour :> ([[0,1],[0,1],[0,1],[0,1],[0,1]]) --> [[0, 1], [0, 1], [0, 1], [0, 1], [0, -1]] (<class 'list'>)


In [7]:
# ○ 成功著陸的，油料剩越多的越好。
# ○ 一個油料耗盡一個墜毀哪個好？墜毀時的速度減到+-40 以內需要多少油料？
# ○ 油料耗盡時的高度距離與速度以自由落體觸地，然後把速度調整到+-40 以內需要多少油料？
# ○ 根據我之前對 y=100 (flat ground 的海拔高度) 的研究 該 state 來自 log 的最後一 state. 
#   當 Simulating 時，該 state 就是 y 首次跨越 flat ground 高度觸地著陸之時。
# ○ cost 必大於 0。 Score 用油料剩餘量計算，由 fuel0 扣下來可到負值。換算成 cost 就是 
#      cost = fuel0 - score
#   核算，當 score 為 fuel0 時 cost 為 0 無誤； 當 score 為 -fuel0 時 cost 為 2fuel0
#   甚至更高，也無誤。 直接用這個數字，配合 SA 的 Temperature 設定要用 visualized 
#   graphs 來訂定 即可。

def cost_of_state(state: STATE) -> int:
    state = copy.deepcopy(state)  # 不要去動到原來的
    score = None  # 有可能是負的，先給個看得出異常的初值。

    # 先判斷是否觸地。若觸地進一步查看速度合格是否合格。若未觸地就是飛走了或油料耗盡。
    if round(state.y,0) < 100:  # 若僅 state.y <= 100 則 100.45 也會被錯當成已著陸。
        # 觸地了，進一步查看速度是否合格
        if round(state.vspeed,0) >= -40 : # 剛從空中下來觸地， vspeed 一定是負值；墜毀的一定比 -40 小
            # 安全降落
            score = state.fuel
        else:  # 墜毀
            # 模擬噴火把速度降到合格範圍內。所剩油料就是成績。
            while True:
                state.next(0,1)  # 有個調整的過程，不是直接填 4 ！
                if round(state.vspeed,0) >= -40:
                    score = state.fuel
                    break
    else:  # 還在空中，飛走了或油料耗盡。
        # 先模擬 free fall 觸地。
        while True: 
            state.next(0,-1)  # 半空中開始模擬 free fall 小心別又錯了，不是直接填 0 ！！
            if round(state.y,0) < 100: break
        # 然後噴火把速度降到合格範圍內，所剩油料就是成績。
        while True:
            if round(state.vspeed,0) >= -40: # 剛從空中下來觸地， vspeed 一定是負值；墜毀的一定比 -40 小。
                # 達標了
                score = state.fuel
                break
            else: # 未達標，繼續噴火把負的 vspeed 速度降到合格範圍內
                state.next(0,1)
    cost = fuel0 - score
    return cost 

%f state_list :> ["自由落體"]  cost_of_state :> (pop()) -->
%f state_list :> ["飛向太空"]  cost_of_state :> (pop()) -->
%f state_list :> ["安全降落"]  cost_of_state :> (pop()) -->
%f state_list :> ["更成功"]   cost_of_state :> (pop()) --> cr
        

state_list :> ["自由落體"]  cost_of_state :> (pop()) --> 1366 (<class 'int'>)
state_list :> ["飛向太空"]  cost_of_state :> (pop()) --> 1934 (<class 'int'>)
state_list :> ["安全降落"]  cost_of_state :> (pop()) --> 322 (<class 'int'>)
state_list :> ["更成功"]   cost_of_state :> (pop()) --> 288 (<class 'int'>)



In [8]:
# 模擬試跑 genome. Genome 可能太長或不夠，傳回訂正好的 genome 以及最後 state  
def simulate_of_genome(input_genome: GENOME) -> Tuple[GENOME,STATE]:
    genome = copy.deepcopy(input_genome)  # make a copy to isolate the original
    state = State(**state0.__dict__)  # init state for simulation
    # 照 genome 跑到觸地、飛走、或油料耗盡時為止， genome 可能含有無效的長度要 cut 掉以免浪費時間嘗試。
    for i in range(len(genome)): 
        state.next(*genome[i])  # run 1 step 
        if round(state.y,0) < 100 or state.y > 3000 or state.fuel <= 0:  # 這高度、油料不能再繼續了
            genome = genome[:i+1]  # 太長的尾巴切掉
            break
        
    # 接續以上跑完 input genome 之後的 state 看看 input genome 是否太短需要補綴 genes 讓 lander 觸地。
    while round(state.y,0) >= 100 and state.y <= 3000 and state.fuel > 0:
        genome.append(genome[-1][:])  # 注意！若不用 [:] slice copy 則 append 進去的都是同一個。gene 是個 shallow array 無誤，可以用 slice copy.        
        state.next(*genome[-1])  # 簡單照抄最後一 step
        
    return genome,state  # 這個 return genome 是 input genome 的訂正版

# 很妙，這產生「自由落體」的 genome 與最後 state 
genome, last_state = simulate_of_genome([[0,0]])
%f cr ( 「自由落體」 ) genome -->
%f last_state -->
%f last_state cost_of_state :> (pop()) -->
    
# 而這產生「全速飛走」的 genome 與最後 state 
genome, last_state = simulate_of_genome([[0,1]])
%f cr ( 「全速飛走」 ) genome -->
%f last_state -->
%f last_state cost_of_state :> (pop()) --> cr


cr ( 「自由落體」 ) genome --> [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]] (<class 'list'>)
last_state --> {'step': 36, 'x': 0, 'y': 95.2720000000003, 'hspeed': 0, 'vspeed': -133.59599999999998, 'fuel': 500, 'angle': 0, 'power': 0, 'force': -3.711} (<class '__main__.State'>)
last_state cost_of_state :> (pop()) --> 1366 (<class 'int'>)

cr ( 「全速飛走」 ) genome --> [[0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 

In [9]:
# fitness function 
# https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)   _*_
verbose = False 
def sa(initial: GENOME) -> Tuple[GENOME,int,int] :
    current = copy.deepcopy(initial)  # 咱 deep 不能用 [:]
    current, current_last_state = simulate_of_genome(current)
    current_cost = cost_of_state(current_last_state)
    
    temp = 17.0  # (1/50)*(2*fuel0)  # <---- [ ] Hyperparameter, 來自 Einstein's problem 的心得，取總分的 1/15 
    num_iterations = 0

    for i in range(1000):  # 若想安全降落為止，可能不如有限次數換個 epoch 試試運氣
        neighbour = get_random_neighbour(current)
        neighbour, neighbour_last_state = simulate_of_genome(neighbour)
        neighbour_cost = cost_of_state(neighbour_last_state)
        
        cost_delta = neighbour_cost - current_cost
        if cost_delta < 0:  # neighbour 較好，都接受。
            current, current_cost, current_last_state = neighbour, neighbour_cost, neighbour_last_state
            if verbose: print({"cost":current_cost, "delta":cost_delta, "temperature":round(temp,2), "exp": "n/a"})
        elif cost_delta >= 0:  # 雖然 neighbour 不好，也偶而接受。
            exp = math.exp(-cost_delta/temp) 
            if random.random() < exp:
                current, current_cost, current_last_state = neighbour, neighbour_cost, neighbour_last_state
                if verbose: print({"cost":current_cost, "delta":cost_delta, "temperature":round(temp,2), "exp": round(exp,2)})
        else:  # 排除相等的狀況，不要浪費時間、debug 版面 <-- 錯！成績相等時 genome 不一定相等。
               # 「相同」應該從 get_random_neighbour() 著手徹底排除。          
            assert False, 'Impossible case' 

        num_iterations += 1
        # if num_iterations % 10 == 0 and temp > 0.1:  # <---- [ ] Hyperparameter
        #     temp -= 0.1  # <---- [ ] Hyperparameter
        if abs(round(current_last_state.vspeed,0)) <= 40 and round(current_last_state.y,0) < 100:
            temp = max(temp - 0.1,0) # 根據觀察 log 決定設計成約數十次到一百次之內降到零度
            if verbose: print('B i n g o ! iterations:{} temperature:{} curent_cost:{}'.format(num_iterations, round(temp,3), current_cost))
            if temp <= 0.01: break
        
    return current, num_iterations, current_cost, current_last_state

In [10]:
def epoch(initial: GENOME=None):

    genome: GENOME = [];
    
    # 有指定就用指定的，否則就用亂數產生一組啟始 genome
    # random.seed(100)
    if initial:  
        genome = copy.deepcopy(initial); 
    else:  
        for i in range(random.randint(1,100)): # 沒看過 genome 長度超過 100 的，多了無所謂會被自動切掉。
            genome.append([0,random.randint(-1, 1)]) 

    genome, iterations, cost, state = sa(genome)

    return (genome, iterations, cost, state)

In [20]:
%f [] constant history 

if __name__ == "__main__":
    best = epoch([[0,0]])  # (genome, iterations, cost, state)
    for i in range(10000):
        result = epoch(best[0])  # (genome, iterations, cost, state)
        if result[2] < best[2]:
            best = result
        if i % 50 == 0 :
            print('Epoch {} best cost {}'.format(i,best[2]))
        %f best history :: append(pop())
    print('Epoch {} best cost {}'.format(i,best[2]))
    print('Done!')


reDef history
Epoch 0 best cost 216
Epoch 50 best cost 190
Epoch 100 best cost 175
Epoch 150 best cost 175
Epoch 200 best cost 175
Epoch 250 best cost 175
Epoch 300 best cost 175
Epoch 350 best cost 175
Epoch 400 best cost 175
Epoch 450 best cost 175
Epoch 500 best cost 175
Epoch 550 best cost 175
Epoch 600 best cost 175
Epoch 650 best cost 175
Epoch 700 best cost 175
Epoch 750 best cost 175
Epoch 800 best cost 175


KeyboardInterrupt: 

In [ ]:
# 取 best result 的絕對 genome 去 web site 跑跑看， 500 - 最佳 cost 175 = 325 與結果一致無誤。  
%f best :> [0] genome_to_command_list :> (pop()) --> 

In [21]:
%%f
\ 這個實驗看來成績 cost 相同的 genome 都一樣！
history ( genome, iterations, cost, state ) <py>
    history = pop()
    distribution = {}
    for i in history:
        try:
            distribution[i[2]] += 1
        except:
            distribution.update({i[2]: 1})
    print(distribution)
    assert False
    print('220','-'*30)
    for i in history:
        if i[2] == 220:
            print(i[0])
    print('221','-'*30)
    for i in history:
        if i[2] == 201:
            print(i[0])
    print('198','-'*30)
    for i in history:
        if i[2] == 198:
            print(i[0])
    print('187','-'*30)
    for i in history:
        if i[2] == 187:
            print(i[0])
    print('186','-'*30)
    for i in history:
        if i[2] == 186:
            print(i[0])
    print('183','-'*30)
    for i in history:
        if i[2] == 183:
            print(i[0])
</py> 


{216: 10, 198: 17, 190: 35, 175: 784}

Failed in </py> (compiling=False): 
Body:
    history = pop()
    distribution = {}
    for i in history:
        try:
            distribution[i[2]] += 1
        except:
            distribution.update({i[2]: 1})
    print(distribution)
    assert False
    print('220','-'*30)
    for i in history:
        if i[2] == 220:
            print(i[0])
    print('221','-'*30)
    for i in history:
        if i[2] == 201:
            print(i[0])
    print('198','-'*30)
    for i in history:
        if i[2] == 198:
            print(i[0])
    print('187','-'*30)
    for i in history:
        if i[2] == 187:
            print(i[0])
    print('186','-'*30)
    for i in history:
        if i[2] == 186:
            print(i[0])
    print('183','-'*30)
    for i in history:
        if i[2] == 183:
            print(i[0])



In [25]:

if __name__ == "__main__":
    history = []
    for i in range(5000):
        result = epoch()  # (genome, iterations, cost, state)
        history.append(result)
        if i % 100 == 0 : print(i, end=" ")
    print('Done!')
    distribution = {}
    for i in history:
        try:
            distribution[i[2]] += 1
        except:
            distribution.update({i[2]: 1})
    print(distribution)


0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 Done!
{223: 215, 201: 261, 269: 19, 249: 94, 242: 102, 194: 157, 198: 188, 246: 74, 183: 71, 214: 32, 254: 22, 205: 218, 186: 163, 275: 38, 216: 190, 217: 74, 213: 91, 244: 7, 212: 276, 229: 15, 227: 140, 253: 94, 235: 102, 211: 13, 209: 186, 199: 51, 190: 159, 224: 93, 238: 113, 261: 41, 210: 53, 191: 40, 257: 58, 272: 32, 204: 4, 268: 42, 179: 33, 206: 69, 220: 164, 346: 2, 231: 118, 232: 45, 187: 53, 225: 17, 316: 6, 218: 20, 202: 102, 290: 16, 294: 13, 236: 31, 195: 31, 228: 58, 320: 3, 239: 60, 265: 27, 250: 45, 240: 12, 291: 5, 283: 11, 298: 8, 264: 65, 258: 21, 301: 12, 251: 12, 288: 5, 237: 7, 243: 35, 335: 2, 203: 20, 215: 2, 188: 6, 219: 5, 267: 4, 259: 6, 285: 1, 208: 3, 331: 1, 226: 6, 310: 2, 221: 36, 222: 9, 184: 4, 247: 26, 233: 12, 296: